In [ ]:
# import torch
# from cvae_utilities import *

# # Define dimensions for testing
# state_dim = 96*96  # Dimensionality of the state input
# action_dim = 2  # Dimensionality of the action input
# latent_dim_state = 50  # Dimensionality of the latent space for states
# latent_dim_action = 2  # Dimensionality of the latent space for actions (identity here)
# posterior_dim = 64  # Dimensionality of the posterior latent space

# # Create dummy data
# batch_size = 64  # Number of samples in a batch
# state = torch.randn(batch_size, state_dim)  # Random state tensor
# action = torch.randn(batch_size, action_dim)  # Random action tensor

# # Initialize the model
# model = CVAEWithDefaultNetwork(
#     state_dim=state_dim,
#     action_dim=action_dim,
#     latent_dim_state=latent_dim_state,
#     latent_dim_action=latent_dim_action,
#     posterior_dim=posterior_dim
# )

# # Test the forward pass
# reconstructed_action, z_mu, z_logvar, _zs = model(state, action)

# # Print the outputs
# # print("Reward shape:", reward.shape)
# # print("Reward values:", reward)
# print("_zs (next state) shape:", _zs.shape)
# # print("_zs values:", _zs)
# # print("Fused latent shape:", fused_latent.shape)
# # print("Fused latent values:", fused_latent)
# # print("Reconstructed state shape:", reconstructed_state.shape)
# # print("Reconstructed state values:", reconstructed_state)
# print("Reconstructed action shape:", reconstructed_action.shape)
# # print("Reconstructed action values:", reconstructed_action)
# print("z_mu shape:", z_mu.shape)
# # print("z_mu values:", z_mu)
# print("z_logvar shape:", z_logvar.shape)




_zs (next state) shape: torch.Size([64, 114])
Reconstructed action shape: torch.Size([64, 2])
z_mu shape: torch.Size([64, 64])
z_logvar shape: torch.Size([64, 64])


In [11]:
from urllib.request import urlopen
from PIL import Image
import timm
from torchvision import transforms
import torch
import torch.nn as nn

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))


In [12]:
model = timm.create_model(
    'resnet18.a1_in1k',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = transforms.Compose([
    transforms.Resize((96, 96)),  # Ensure input is 96x96
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

output = model(transforms(img).unsqueeze(0))  # output is (batch_size, num_features) shaped tensor

# or equivalently (without needing to set num_classes=0)

output = model.forward_features(transforms(img).unsqueeze(0))
# output is unpooled, a (1, 512, 7, 7) shaped tensor

output = model.forward_head(output, pre_logits=True)
# output is a (1, num_features) shaped tensor

In [ ]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

# Set up the dataset.
delta_timestamps = {
    # Load the previous image and state at -0.1 seconds before current frame,
    # then load current image and state corresponding to 0.0 second.
    "observation.image": [0.0],
    "observation.state": [0.0],
    # Load the previous action (-0.1), the next action to be executed (0.0),
    # and 14 future actions with a 0.1 seconds spacing. All these actions will be
    # used to supervise the policy.
    "action": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3],
}

dataset = LeRobotDataset("lerobot/pusht", delta_timestamps=delta_timestamps)


Fetching 212 files: 100%|██████████| 212/212 [00:00<00:00, 12114.50it/s]


In [17]:
# Create dataloader for offline training.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=4,
    batch_size=64,
    shuffle=True,
    # pin_memory=device != torch.device("cpu"),
    drop_last=True,
)

In [51]:
np.arange(1,7).repeat(2).reshape(6,2).reshape(1,-1)[0,:6]

array([1, 1, 2, 2, 3, 3])

In [18]:
for batch in dataloader:
    # batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
    print(batch.keys())
    print(batch["observation.image"].shape)
    print(batch["observation.state"].shape)
    print(batch["action"].shape)
    break

dict_keys(['observation.image', 'observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.reward', 'next.done', 'next.success', 'index', 'observation.image_is_pad', 'observation.state_is_pad', 'action_is_pad'])
torch.Size([64, 1, 3, 96, 96])
torch.Size([64, 1, 2])
torch.Size([64, 10, 2])


In [ ]:
# example.py
import gymnasium as gym
import gym_pusht

env = gym.make("gym_pusht/PushT-v0", obs_type="pixels", render_mode="rgb_array")
observation, info = env.reset()
transforms = transforms.Compose([
    transforms.Resize((96, 96)),  # Ensure input is 96x96
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    print(observation.shape)
    image = env.render()

    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [166]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import theseus as th
import copy
from cvae_utilities import *

class CVAEWithTrajectoryOptimization(nn.Module):
    def __init__(self, cfg):
        super(CVAEWithTrajectoryOptimization, self).__init__()
        state_dim = cfg.state_dim
        action_dim = cfg.action_dim * cfg.horizon
        latent_dim_state = cfg.latent_dim_state 
        latent_dim_action = cfg.latent_dim_action * cfg.horizon
        posterior_dim = cfg.posterior_dim
        device = cfg.device
        self.state_encoder = StateEncoder(state_dim, latent_dim_state).to(device)
        self.action_encoder = ActionEncoder(action_dim, latent_dim_action).to(device)
        self.fusing_encoder = FusingEncoder(latent_dim_state, latent_dim_action, posterior_dim).to(device)
        self.dynamics_function = DynamicsFunction(latent_dim_state, latent_dim_action, posterior_dim).to(device)
        self.action_decoder = ActionDecoder(posterior_dim, latent_dim_state, action_dim).to(device)
        self.reward_decoder = RewardDecoder(latent_dim_state, latent_dim_action, posterior_dim).to(device)
        self.posterior_dim = posterior_dim
        self.device = device
        self.cfg = cfg

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, zs, za):
        """
        Perform a forward pass through the model.
        Args:
            obs: Observation (raw img input), shape [batch_size, state_dim].
            state: State (latent state input), shape [batch_size, state_dim].
            action: Action (latent action input), shape [batch_size, horizon * action_dim].
        Returns:
            reconstructed_action: Reconstructed action, shape [batch_size, horizon, action_dim].
            z_mu: Mean of the posterior Gaussian, shape [batch_size, posterior_dim].
            z_logvar: Log-variance of the posterior Gaussian, shape [batch_size, posterior_dim].
            _zs: Predicted next latent state, shape [batch_size, state_dim].
        """
    
        # Encode state and action
        # zs = self.state_encoder(obs, state)
        # za = self.action_encoder(action)

        # Fuse state and action for posterior Gaussian
        z_mu, z_logvar = self.fusing_encoder(zs, za)

        # Sample from posterior Gaussian
        fused_latent = self.reparameterize(z_mu, z_logvar)

        reconstructed_action = self.action_decoder(fused_latent, zs)

        # Predict next latent state
        _zs = self.dynamics_function(zs, self.action_encoder(reconstructed_action), fused_latent)#TODO: check if this is correct

        return reconstructed_action, z_mu, z_logvar, _zs
    
    def generate_action(self, obs, state, z):
        """
        Generate an action based on the given observation.
        Args:
            obs: Observation (raw state input), shape [batch_size, state_dim].
            state: State (latent state input), shape [batch_size, state_dim].
            z: Latent variable z, shape [batch_size, posterior_dim].
        Returns:
            action: Generated action, shape [batch_size, action_dim].
        """
        # Ensure observation is a PyTorch tensor
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float32, device=self.device)
        if not isinstance(state, torch.Tensor):
            state = torch.tensor(state, dtype=torch.float32, device=self.device)


        if obs.ndim == 1:
            obs = obs.unsqueeze(0)  # Add batch dimension if necessary
        if state.ndim == 1:
            state = state.unsqueeze(0)  # Add batch dimension if necessary

        # Encode observation into latent state
        zs = self.state_encoder(obs, state)  # Encoded latent state

        # # Sample latent variable z (assuming zero-mean Gaussian prior)
        # z = torch.randn(obs.shape[0], self.posterior_dim, device=self.device)  # Prior sample for z

        # Decode action from latent state and z
        action = self.action_decoder(z, zs)  # Reconstruct action

        return action

    def plan_with_theseus_update(self, obs, state, action, horizon, gamma, model, cfg, eval_mode=False):
        """
        Perform trajectory optimization using Theseus.
        Args:
            obs: Observation (raw state input).
            horizon: Planning horizon.
            gamma: Discount factor.
            model: The CVAE model containing dynamics and state encoders.
            cfg: Configuration object (for damping, step size, etc.).
            eval_mode: Whether to evaluate without gradients.
        """
        # Prepare initial observation
        batch_size = obs.shape[0]
        obs = torch.tensor(obs[:,0,...], dtype=torch.float32, device=self.device)#.unsqueeze(0)  # [1, obs_dim]
        state = torch.tensor(state[:,0,...], dtype=torch.float32, device=self.device)#.unsqueeze(0)  # [1, state_dim]
        horizon = int(min(horizon, cfg.horizon))  # Clip planning horizon

        # Initialize latent state and actions
          # Latent state
        # actions = torch.zeros(batch_size, horizon, cfg.action_dim, device=self.device, requires_grad=True)  # Initial actions

        # Precompute initial actions using policy (if available)
        pi_actions = torch.empty(batch_size, horizon, cfg.action_dim * horizon, device=self.device)
        zs = model.state_encoder(obs, state)
        zs0 = zs.clone()
        # pz = torch.randn(batch_size, model.posterior_dim, device=self.device)
        # pz0 = pz.clone()
        # with torch.no_grad():
        # consistance_loss = 0 #TODO: calculate consistance loss by mse(_zs) KL(zp|N(0,1))
        for t in range(horizon):
            # pi_actions[:,t] = model.generate_action(obs, state, pz)
            za = model.action_encoder(action[:,2*t:2*(horizon+t)])
            reconstructed_action, z_mu, z_logvar, (zs0, zp) = model(zs0, za)
            pi_actions[:,t] = reconstructed_action
        init_actions = pi_actions.view(batch_size, -1)  # Initial actions for Theseus

        # Define cost function
        def value_cost_fn(optim_vars, aux_vars):
            actions = optim_vars[0].tensor  # [1, horizon * action_dim]
            print(actions.shape)
            actions = actions.view(1, horizon, cfg.action_dim * horizon)
            obs = aux_vars[0].tensor
            state = aux_vars[1].tensor
            # pz = aux_vars[2].tensor
            z = model.state_encoder(obs, state)  # Latent state
            total_reward = 0.0
            discount = 1.0
            pz = torch.randn(batch_size, model.posterior_dim, device=self.device)
            # Compute cumulative reward
            for t in range(horizon):
                reward = model.reward_decoder(z, actions[:,t], pz)
                z, pz = model.dynamics_function(z, actions[:,t], pz)
                total_reward += discount * reward
                discount *= gamma
            err = -total_reward.nan_to_num_(0) + 1e3
            return err

        # Set up Theseus optimization
        # actions_vars = []
        # obs_vars = []
        # state_vars = []
        # for i in range(batch_size):
        #     actions_vars.append(th.Variable(init_actions[i], name="actions"))
        #     obs_vars.append(th.Variable(obs[i], name="obs"))
        #     state_vars.append(th.Variable(state[i], name="state"))
        objective = th.Objective()
        theseus_inputs = {}
        for i in range(batch_size):
            actions_var = th.Variable(init_actions[i], name=f"actions_{i}")
            obs_var = th.Variable(obs[i], name=f"obs_{i}")
            state_var = th.Variable(state[i], name=f"state_{i}")
            cost_function = th.AutoDiffCostFunction(
                [actions_var],
                value_cost_fn,
                dim=0,
                aux_vars=[obs_var, state_var],#, posterior],
                name=f"value_cost_fn{i}",
            )
            objective.add(cost_function)
            theseus_inputs.update({
                f"actions_{i}": init_actions[i],
                f"obs_{i}": obs[i],
                f"state_{i}": state[i]
            })
        # print(theseus_inputs)
        # posterior = th.Variable(pz0, name="posterior")

        # cost_function = th.AutoDiffCostFunction(
        #     [actions_var],
        #     value_cost_fn,
        #     dim=1,
        #     aux_vars=[obs_var, state_var],#, posterior],
        #     name="value_cost_fn",
        # )
        
        # objective.add(cost_function)
        optimizer = th.LevenbergMarquardt(
            objective,
            th.CholeskyDenseSolver,
            max_iterations=cfg.max_iterations,
            step_size=cfg.step_size,
        )
        theseus_layer = th.TheseusLayer(optimizer)
        theseus_layer.to(device=self.device)

        # theseus_inputs = {"actions": init_actions, "obs": obs, "state": state}#, "posterior": pz0}

        # Solve optimization problem
        updated_inputs, info = theseus_layer.forward(
            theseus_inputs,
            optimizer_kwargs={
                "track_best_solution": True,
                "damping": cfg.damping,
                "verbose": True,
            },
        )
        best_actions = info.best_solution["actions"].view(horizon, -1)

        return best_actions

# Example Configuration
class Config:
    def __init__(self):
        self.horizon = 7
        self.action_dim = 2
        # Theseus optimization parameters
        self.max_iterations = 100
        self.step_size = 1e-2
        self.damping = 1e-3
        self.min_std = 1e-4
        self.discount = 0.99
        self.state_dim = 512*3*3 + 2

        self.latent_dim_state = 50
        self.latent_dim_action = self.action_dim
        self.posterior_dim = 64
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [167]:
# Example Usage
if __name__ == "__main__":
    # Config and hyperparameters
    cfg = Config()
    

    # Create CVAE model
    model = CVAEWithTrajectoryOptimization(cfg)

    # Example observation
    obs = torch.randn(64,1,3,96,96).to(device)
    state = torch.randn(64,1,2).to(device)
    action = torch.randn(64,28).to(device)
    # obs = (obs, state)

    # Perform trajectory optimization with Theseus
    optimized_actions = model.plan_with_theseus_update(obs, state, action, horizon=cfg.horizon, gamma=cfg.discount, model=model, cfg=cfg)
    print("Optimized Actions:", optimized_actions)


/tmp/ipykernel_139914/2686813838.py:109: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs = torch.tensor(obs[:,0,...], dtype=torch.float32, device=self.device)#.unsqueeze(0)  # [1, obs_dim]
/tmp/ipykernel_139914/2686813838.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state[:,0,...], dtype=torch.float32, device=self.device)#.unsqueeze(0)  # [1, state_dim]


AttributeError: 'Variable' object has no attribute 'dof'